# Capítulo 7
## _Limpeza e preparação de dados_

In [235]:
import numpy as np
import pandas as pd
import re
from numpy import nan as NA
from pandas import Series, DataFrame

- Tratando dados ausentes

In [236]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [237]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [238]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

- Filtrando dados ausentes

In [239]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [240]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], 
                     [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [241]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [242]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [243]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [244]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [245]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = NA # Todos os elementos de 0 até 4 e 2 nas 
df.iloc[:2, 2] = NA # colunas 1 e 2 se tornarão null
df

,0,1,2
0,-0.036624,NaN,NaN
1,-0.484280,NaN,NaN
2,0.238140,NaN,-0.981498
3,-0.488734,NaN,-1.278592
4,0.193096,-1.298301,-2.026822
5,-1.224612,-0.840313,-1.135108
6,-0.344502,-0.103765,-2.714114


In [246]:
df.dropna()

,0,1,2
4,0.193096,-1.298301,-2.026822
5,-1.224612,-0.840313,-1.135108
6,-0.344502,-0.103765,-2.714114


In [247]:
df.dropna(thresh=2) # preservo os elementos do índice 2 em diante

,0,1,2
2,0.238140,NaN,-0.981498
3,-0.488734,NaN,-1.278592
4,0.193096,-1.298301,-2.026822
5,-1.224612,-0.840313,-1.135108
6,-0.344502,-0.103765,-2.714114


- Preenchendo dados ausentes

In [248]:
df.fillna(0)

,0,1,2
0,-0.036624,0.000000,0.000000
1,-0.484280,0.000000,0.000000
2,0.238140,0.000000,-0.981498
3,-0.488734,0.000000,-1.278592
4,0.193096,-1.298301,-2.026822
5,-1.224612,-0.840313,-1.135108
6,-0.344502,-0.103765,-2.714114


In [249]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.036624,0.500000,0.000000
1,-0.484280,0.500000,0.000000
2,0.238140,0.500000,-0.981498
3,-0.488734,0.500000,-1.278592
4,0.193096,-1.298301,-2.026822
5,-1.224612,-0.840313,-1.135108
6,-0.344502,-0.103765,-2.714114


In [250]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.036624,0.000000,0.000000
1,-0.484280,0.000000,0.000000
2,0.238140,0.000000,-0.981498
3,-0.488734,0.000000,-1.278592
4,0.193096,-1.298301,-2.026822
5,-1.224612,-0.840313,-1.135108
6,-0.344502,-0.103765,-2.714114


In [251]:
df2 = pd.DataFrame(np.random.randn(6,3))
df2.iloc[2:, 1] = NA
df2.iloc[4:, 2] = NA
df2

,0,1,2
0,-0.512338,1.342810,1.491791
1,-1.504917,-1.582262,-2.489750
2,-2.011076,NaN,0.337476
3,-0.131013,NaN,2.299629
4,0.395416,NaN,NaN
5,-0.608508,NaN,NaN


In [252]:
df2.fillna(method='ffill')

,0,1,2
0,-0.512338,1.342810,1.491791
1,-1.504917,-1.582262,-2.489750
2,-2.011076,-1.582262,0.337476
3,-0.131013,-1.582262,2.299629
4,0.395416,-1.582262,2.299629
5,-0.608508,-1.582262,2.299629


In [253]:
df2.fillna(method='ffill', limit=2)

,0,1,2
0,-0.512338,1.342810,1.491791
1,-1.504917,-1.582262,-2.489750
2,-2.011076,-1.582262,0.337476
3,-0.131013,-1.582262,2.299629
4,0.395416,NaN,2.299629
5,-0.608508,NaN,2.299629


## _Transformação de dados_

- Removendo duplicatas

In [254]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [255]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [256]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [257]:
data['v1'] = range(7)
data.drop_duplicates(['k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


In [258]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


- Transformando dados usando uma função ou um mapeamento

In [259]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [260]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

In [261]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


- Substituindo valores

In [262]:
data = pd.Series([1, -999, 2, -999, -1000, 3, 45, 9, 5, 45, -50])
data

0        1
1     -999
2        2
3     -999
4    -1000
5        3
6       45
7        9
8        5
9       45
10     -50
dtype: int64

In [263]:
data.replace(-999, np.nan)

0        1.0
1        NaN
2        2.0
3        NaN
4    -1000.0
5        3.0
6       45.0
7        9.0
8        5.0
9       45.0
10     -50.0
dtype: float64

In [264]:
data.replace([-999, -1000], np.nan)

0      1.0
1      NaN
2      2.0
3      NaN
4      NaN
5      3.0
6     45.0
7      9.0
8      5.0
9     45.0
10   -50.0
dtype: float64

In [265]:
data.replace([45, -50], [np.nan, 0])

0        1.0
1     -999.0
2        2.0
3     -999.0
4    -1000.0
5        3.0
6        NaN
7        9.0
8        5.0
9        NaN
10       0.0
dtype: float64

In [266]:
data.replace({-999: np.nan, 45: 0})

0        1.0
1        NaN
2        2.0
3        NaN
4    -1000.0
5        3.0
6        0.0
7        9.0
8        5.0
9        0.0
10     -50.0
dtype: float64

- Renomeando os índices dos eixos

In [267]:
data = pd.DataFrame(np.arange(12).reshape((3,4)), 
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [268]:
def transform(x):
    return x[:4].upper()

#data.index.map(transform)
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [269]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [270]:
data.rename(index={'OHIO': 'Indiana'}, 
            columns={'three' : 3})

,one,two,3,four
Indiana,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- Discretização e Compartimentalização (binning)

In [271]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
categorical = pd.cut(ages, bins)
categorical

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [272]:
categorical.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [273]:
categorical.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [274]:
pd.value_counts(categorical)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

In [275]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [276]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [277]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.23, 0.44], (0.66, 0.88], (0.66, 0.88], (0.44, 0.66], (0.009, 0.23], ..., (0.66, 0.88], (0.23, 0.44], (0.009, 0.23], (0.23, 0.44], (0.23, 0.44]]
Length: 20
Categories (4, interval[float64, right]): [(0.009, 0.23] < (0.23, 0.44] < (0.44, 0.66] < (0.66, 0.88]]

In [278]:
data = np.random.randn(1000)
categorical = pd.qcut(data, 4)
categorical

[(-0.78, -0.0656], (-0.0656, 0.6], (-0.78, -0.0656], (-3.0949999999999998, -0.78], (0.6, 3.894], ..., (0.6, 3.894], (-0.78, -0.0656], (-0.78, -0.0656], (-3.0949999999999998, -0.78], (-3.0949999999999998, -0.78]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.0949999999999998, -0.78] < (-0.78, -0.0656] < (-0.0656, 0.6] < (0.6, 3.894]]

In [279]:
pd.value_counts(categorical)

(-3.0949999999999998, -0.78]    250
(-0.78, -0.0656]                250
(-0.0656, 0.6]                  250
(0.6, 3.894]                    250
Name: count, dtype: int64

In [280]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

[(-1.398, -0.0656], (-0.0656, 1.257], (-1.398, -0.0656], (-3.0949999999999998, -1.398], (-0.0656, 1.257], ..., (-0.0656, 1.257], (-1.398, -0.0656], (-1.398, -0.0656], (-1.398, -0.0656], (-1.398, -0.0656]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.0949999999999998, -1.398] < (-1.398, -0.0656] < (-0.0656, 1.257] < (1.257, 3.894]]

- Detectando e filtrando valores discrepantes

In [281]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.032778,-0.052520,-0.035885,-0.002022
std,0.988103,0.997335,0.994965,0.978971
min,-2.910662,-3.019376,-2.842447,-3.441819
25%,-0.612504,-0.707041,-0.674007,-0.635974
50%,0.057458,-0.047876,-0.061495,0.029664
75%,0.715283,0.608811,0.605189,0.669627
max,3.360897,3.224034,3.424722,3.023134


In [282]:
# encontrar os valores absolutos maiores do que 3 em alguma das colunas
col = data[2]
col[np.abs(col) > 3]

190    3.424722
263    3.030593
370    3.037328
433    3.105988
Name: 2, dtype: float64

In [283]:
data[(np.abs(data) > 3).any(axis='columns')]

,0,1,2,3
181,0.092509,-3.019376,-0.534652,1.155369
190,0.927954,0.189540,3.424722,0.871550
263,0.236777,-0.716685,3.030593,-0.836930
355,-0.299858,-0.381925,-1.464694,-3.441819
370,1.160712,0.463088,3.037328,0.312797
433,0.474603,-0.558537,3.105988,2.027675
610,-1.037695,1.534879,0.659497,3.023134
747,0.208767,3.224034,-1.113995,0.493246
968,3.360897,-0.276134,0.091517,0.605735


In [284]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.032417,-0.052724,-0.036484,-0.001604
std,0.986952,0.996565,0.993037,0.977445
min,-2.910662,-3.000000,-2.842447,-3.000000
25%,-0.612504,-0.707041,-0.674007,-0.635974
50%,0.057458,-0.047876,-0.061495,0.029664
75%,0.715283,0.608811,0.605189,0.669627
max,3.000000,3.000000,3.000000,3.000000


In [285]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,1.0,1.0
1,1.0,1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,-1.0
4,-1.0,-1.0,1.0,1.0


- Permutação e amostragem aleatória

In [286]:
df = pd.DataFrame(np.arange(20).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([4, 3, 1, 2, 0])

In [287]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [288]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3


In [289]:
df.sample(n=2)

,0,1,2,3
2,8,9,10,11
1,4,5,6,7


In [290]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

4    4
0    5
2   -1
4    4
1    7
1    7
2   -1
4    4
1    7
4    4
dtype: int64

- Calculando variáveis indicadoras/dummy

In [291]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'], dtype=int)

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [292]:
dummies = pd.get_dummies(df['key'], prefix='key', dtype=int)
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [293]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', 
                       header=None, names=mnames)
movies[:15]

C:\Users\Candido\AppData\Local\Temp\ipykernel_3432\2379743331.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [294]:
#extraimos a lista de gêneros únicos do conjunto de dados
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))

genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [295]:
# Construindo um DataFrame indicador a parti de um DF contendo apenas zeros
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [296]:
# Intetando pelos filmes e definindo entradas para cada linha de dunnies com 1
gen = movies.genres[0]
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1
    
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[:10]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [297]:
#np.random.seed(12345)
values = np.random.rand(10)
values

array([0.01181546, 0.5432266 , 0.4797235 , 0.46472416, 0.51443613,
       0.261958  , 0.72673777, 0.82334412, 0.24371291, 0.69939202])

In [298]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins), dtype=float)

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,1.0
8,0.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0


## _Manipulação de strings_

- Expressões regulares

In [299]:
text = "foo    bar\t baz   \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [300]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [301]:
regex.findall(text)

['    ', '\t ', '   \t']

In [302]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"
regex = re.compile(pattern, flags=re.IGNORECASE)
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [303]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [304]:
text[m.start():m.end()]

'dave@google.com'

In [305]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [306]:
# segmentar o endereçp de e-mail em três partes: 
# nome do usuário, nome do domínio e sufixo do domínio
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('clqb@academico.ufpb.br')
m.groups()

('clqb', 'academico.ufpb', 'br')

In [307]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [308]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



- Funções strings vetorizadas no pandas

In [309]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [310]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [311]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [312]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [313]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

In [314]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [315]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN
